## valuma-alue työkalu avoimeen lähdekoodiin

###  Ympäristö

In [2]:
import geopandas as gpd
from qgis.core import *
from qgis.analysis import QgsNativeAlgorithms
from qgis.utils import *
import os, sys,tempfile,requests,json
import matplotlib.pyplot as plt
from qgis.core import QgsProcessingMultiStepFeedback
from osgeo import gdal,gdal_array

QgsApplication.setPrefixPath(r'C:\OSGeo4W\apps\qgis', True)
qgs = QgsApplication([], False)
qgs.initQgis()
sys.path.append(r'C:\OSGeo4W\apps\qgis\python\plugins')
import processing
from processing.core.Processing import Processing
Processing.initialize()

### Aineiston lukeminen

In [3]:
purku = r'C:/Users/mkesala/Desktop/Kompassi_v4d/Kompassi_data.gpkg'
layer='Pistehavainto'
jako5 = QgsVectorLayer("crs='EPSG:3067' crs='EPSG:3067' url='https://paikkatieto.ymparisto.fi/arcgis/rest/services/LAPIO/LAPIO_hydrografia/MapServer/23'","jako5","arcgisfeatureserver")
DEM = "https://aineistot.metsakeskus.fi/metsakeskus/services/Vesiensuojelu/DEM/MapServer/WCSServer"
D8 = "https://aineistot.metsakeskus.fi/metsakeskus/services/Vesiensuojelu/D8_suomi/ImageServer/WCSServer"
snapdist = 5
inp = QgsVectorLayer(purku+"|layername="+layer,"point","ogr")
feedback = QgsProcessingFeedback()

In [4]:
def getBboxWmsFormat(in_feat:QgsVectorLayer):
    desc=in_feat.extent()
    x_min=int(desc.xMinimum())
    y_min=int(desc.yMinimum())
    x_max=int(desc.xMaximum())+1
    y_max=int(desc.yMaximum())+1
    srid=str(in_feat.crs().authid())
    exte = str(x_min)+","+str(y_min)+","+str(x_max)+","+str(y_max)
    witdth = x_max - x_min
    height = y_max - y_min
    
    return exte,srid,witdth,height

In [5]:
def getWater(input_polygon:QgsVectorLayer,taso):
    tempd = tempfile.TemporaryFile()
    tempd = tempd.name+str(taso)+'.tif'
    bbox = getBboxWmsFormat(input_polygon)
    
    ss = bbox[0].split(',')
    
    wmsurl = 'https://aineistot.metsakeskus.fi/metsakeskus/rest/services/Vesiensuojelu/'+taso+'/ImageServer/exportImage?'
    params = {"bbox":str(round(int(ss[0])-100,-1))+","+str(round(int(ss[1])-100,-1))+","+str(round(int(ss[2])+100,-1))+","+str(round(int(ss[3])+100,-1)),
                "bboxSR":3067,
                "size":str((round(int(ss[2])+100,-1)-round(int(ss[0])-100,-1))/2)+","+str((round(int(ss[3])+100,-1) - round(int(ss[1])-100,-1))/2),
                "imageSR":3067,
                "format":'tiff',
                "pixelType":"F32",
                "noData":-9999,
                "noDataInterpretation":"esriNoDataMatchAny",
                "interpolation":"+RSP_BilinearInterpolation",
                "f":"image"}

    try:
        respo= requests.get(wmsurl,params,allow_redirects=True)
        
        if respo.status_code != 200:
           info = "Cannot connect to "+str(taso)+ " data: "+str(wmsurl)
           infolevel = 3
        else:
            open(tempd,'wb').write(respo.content)
    
    except:
        info = "Cannot connect to "+str(taso)+ " data: "+str(wmsurl)
        infolevel = 3
    
        
    try:
        test = gdal.Open(tempd)
        test_b = test.GetRasterBand(1)
        test_a = test_b.ReadAsArray()
        if np.max(test_a) > 1:
            info = str(taso)+" data is ok!"
            infolevel = 1
            del test,test_b,test_a
        else:
           info = "Not able find "+str(taso)+" data from area: "+str(bbox[0])
           infolevel = 3
    except:
        info = "Not able find "+str(taso)+" data from area: "+str(bbox[0])
        infolevel = 3

    return tempd,info,infolevel

In [8]:
class MyFeedBack(QgsProcessingFeedback):

    def setProgressText(self, text):
        print(str(text))
    """
    def pushInfo(self, info):
        print(info)

    def pushCommandInfo(self, info):
        print(info)

    def pushDebugInfo(self, info):
        print(info)

    def pushConsoleInfo(self, info):
        print(info)

    def reportError(self, error, fatalError=False):
        print(error)"""

In [6]:
"""
snap = processing.run("saga:snappointstoraster",
                      {'INPUT':purku+"|layername="+layer,
                       'GRID':DEM,
                       'OUTPUT':'TEMPORARY_OUTPUT',
                       'MOVES':'TEMPORARY_OUTPUT',
                       'DISTANCE':snapdist,
                       'SHAPE':0,
                       'EXTREME':0})
snap = QgsVectorLayer(snap["OUTPUT"],"snap","ogr")
print (snap.featureCount())
"""
jako5_raj = processing.run("native:joinattributesbylocation",
                           {'INPUT':jako5,
                            'PREDICATE':[0],
                            'JOIN':inp,
                            'JOIN_FIELDS':[],
                            'METHOD':1,
                            'DISCARD_NONMATCHING':True,
                            'PREFIX':'',
                            'OUTPUT':'TEMPORARY_OUTPUT'})

jako5_raj = jako5_raj['OUTPUT']
print (jako5_raj.featureCount())

jako5_raj = processing.run("native:buffer",
                           {'INPUT':jako5_raj,
                            'DISTANCE':500,
                            'SEGMENTS':5,
                            'END_CAP_STYLE':0,
                            'JOIN_STYLE':0,
                            'MITER_LIMIT':2,
                            'DISSOLVE':True,
                            'OUTPUT':'TEMPORARY_OUTPUT'})

jako5_raj = jako5_raj['OUTPUT']
print (jako5_raj.featureCount())
#jako5_raj.updateExtents()

1
1


In [9]:
DEM = getWater(jako5_raj,"DEM")
D8= getWater(jako5_raj,"D8_suomi")

C:\Users\mkesala\AppData\Local\Temp\ipykernel_22492\2658332860.py:3: ResourceWarning: unclosed file <_io.BufferedRandom name=4>
  tempd = tempd.name+str(taso)+'.tif'
C:\Users\mkesala\AppData\Local\Temp\ipykernel_22492\2658332860.py:27: ResourceWarning: unclosed file <_io.BufferedWriter name='C:\\Users\\mkesala\\AppData\\Local\\Temp\\tmpd_zipi2rDEM.tif'>
  open(tempd,'wb').write(respo.content)
C:\Users\mkesala\AppData\Local\Temp\ipykernel_22492\2658332860.py:27: ResourceWarning: unclosed file <_io.BufferedWriter name='C:\\Users\\mkesala\\AppData\\Local\\Temp\\tmpd9cooazxD8_suomi.tif'>
  open(tempd,'wb').write(respo.content)


In [15]:
snap = processing.run("saga:snappointstoraster",
                      {'INPUT':QgsVectorLayer(purku+"|layername="+layer,"point","ogr"),
                       'GRID':DEM[0],
                       'OUTPUT':'TEMPORARY_OUTPUT',
                       'MOVES':'TEMPORARY_OUTPUT',
                       'DISTANCE':snapdist,
                       'SHAPE':0,
                       'EXTREME':0})
#snap = QgsVectorLayer(snap["OUTPUT"],"snap","ogr")
print (snap["OUTPUT"])
snap_df = gpd.read_file(snap["OUTPUT"])
print (snap_df)

C:/Users/mkesala/AppData/Local/Temp/processing_qKJPFk/b5b6118aedf347abac6a44cbdfde9b22/OUTPUT.shp
   fid        Nimi Havainnot  Kuva                        geometry
0    1  purkupiste      None  None  POINT (446941.000 7196333.000)


In [12]:
tempd = tempfile.TemporaryFile()
tempd = tempd.name+'.tif'
    
d8_raj = processing.run("grass7:r.reclass",
                        {'input':D8[0],
                         'rules':'',
                         'txtrules':'1=8\n2=7\n4=6\n8=5\n16=4\n32=3\n64=2\n128=1\n',
                         'output':tempd,
                         'GRASS_REGION_PARAMETER':'',
                         'GRASS_REGION_CELLSIZE_PARAMETER':0,
                         'GRASS_RASTER_FORMAT_OPT':'',
                         'GRASS_RASTER_FORMAT_META':''})

print (d8_raj["output"])

C:\Users\mkesala\AppData\Local\Temp\ipykernel_22492\1045503275.py:2: ResourceWarning: unclosed file <_io.BufferedRandom name=4>
  tempd = tempd.name+'.tif'


C:\Users\mkesala\AppData\Local\Temp\tmp7re3yaet.tif


In [18]:
#snap = QgsVectorLayer(snap["OUTPUT"],"snap","ogr")

snapfeat = next(snap.getFeatures())
print (snapfeat.geometry())
print (snapfeat.geometry().asPoint().x())

<QgsGeometry: Point (446941 7196333)>
446941.0


In [20]:
srid=str(snap.crs().authid())
geom = str(snapfeat.geometry().asPoint().x())+","+str(snapfeat.geometry().asPoint().y())+" ["+srid+"]"
print (geom)

446941.0,7196333.0 [EPSG:3067]


In [21]:
basin = processing.run("grass7:r.water.outlet",
                       {'input':'C:/Users/mkesala/AppData/Local/Temp/tmp7re3yaet.tif',
                        'coordinates':geom,
                        'output':'TEMPORARY_OUTPUT',
                        'GRASS_REGION_PARAMETER':None,
                        'GRASS_REGION_CELLSIZE_PARAMETER':0,
                        'GRASS_RASTER_FORMAT_OPT':'',
                        'GRASS_RASTER_FORMAT_META':''})

In [22]:
def raster2vector(in_rast,data):
    """transfrom input buffer zone raster to vector"""
    vectn = processing.run("gdal:polygonize", 
        {'INPUT':in_rast,
        'BAND':1,
        'FIELD':'DN',
        'EIGHT_CONNECTEDNESS':False,
        'EXTRA':'',
        'OUTPUT':"TEMPORARY_OUTPUT"})
    
    vect = QgsVectorLayer(vectn['OUTPUT'],"vyohyke","ogr")
    #arealist = [feat.geometry().area() for feat in vect.getFeatures() if feat['DN']==1]
    #namelist = list(data.columns)
    #for i in namelist:
     #   vect.dataProvider().addAttributes([QgsField(i,QVariant.Double)])
      #  vect.updateFields()
    
    with edit(vect):
        for feat in vect.getFeatures():
            #raster value 0 means out
            if feat['DN'] == 0:
                vect.deleteFeature(feat.id())
            
            #delete small parts
            #if max(arealist) - feat.geometry().area() > max(arealist) /1.4:
                #vect.deleteFeature(feat.id())

            for i in namelist:
                datac = data[[i]]
                print (datac.iloc[0,0])
                feat[i] = float(datac.iloc[0,0])
            
            geom = feat.geometry()
            buffer = geom.buffer(10, 5)
            buffer = buffer.buffer(-10,5)
            feat.setGeometry(buffer)

            vect.updateFeature(feat)
    
    return vect

In [28]:
#vect = raster2vector(basin["output"],{'pinta_ala':30})
#print (vect.source())
vect_df = gpd.read_file(vect.source())
print (vect_df)

    DN                                           geometry
0    1  POLYGON ((446962.108 7196354.108, 446962.000 7...
1    1  POLYGON ((446960.108 7196352.108, 446960.000 7...
2    1  POLYGON ((446958.108 7196350.108, 446958.000 7...
3    1  POLYGON ((446956.108 7196348.108, 446956.000 7...
4    1  POLYGON ((446954.108 7196346.108, 446954.000 7...
5    1  POLYGON ((446952.108 7196344.108, 446952.000 7...
6    1  POLYGON ((446950.108 7196342.108, 446950.000 7...
7    1  POLYGON ((446948.108 7196340.108, 446948.000 7...
8    1  POLYGON ((446946.108 7196338.108, 446946.000 7...
9    1  POLYGON ((446944.108 7196336.108, 446944.000 7...
10   1  POLYGON ((446942.108 7196334.108, 446942.000 7...
11   1  POLYGON ((446940.108 7196332.108, 446940.000 7...
